In [1]:
from datasets import load_dataset, get_dataset_split_names

get_dataset_split_names("curaihealth/medical_questions_pairs")

/home/hxastur/University/ck/medical-questions-answering/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['train']

In [2]:
get_dataset_split_names("ruslanmv/ai-medical-chatbot")

['train']

In [2]:
ds = load_dataset("curaihealth/medical_questions_pairs")

In [ ]:
ds = ds["train"].train_test_split(test_size=0.2, seed=42)

In [4]:
ds["train"][0]

{'dr_id': 4,
 'question_1': 'What are the signs of having frostbite?',
 'question_2': 'What exactly is the treatment for frostbite?',
 'label': 0}

In [5]:
ds["train"][:3]

{'dr_id': [4, 2, 2],
 'question_1': ['What are the signs of having frostbite?',
  "I'm due for my birth control shot on the 24 but I've been having uterus pain on the right side of my uterus I'm not sure what it can be?",
  "If I don't have the egg white cervical musus am I still ovualating?"],
 'question_2': ['What exactly is the treatment for frostbite?',
  'After how long of stopping the birth control shots can one get pregnant?',
  'Is it imperative that one has egg white cervical mucous while ovulating?'],
 'label': [0, 0, 1]}

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 2438
    })
    test: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 610
    })
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [8]:
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

The max length for the tokenizer is: 512


In [ ]:
tokenizer(
    ds["train"][0]["question_1"], ds["train"][0]["question_2"], return_tensors="pt"
)

{'input_ids': tensor([[  101,  2054,  2024,  1996,  5751,  1997,  2383, 10097, 16313,  2063,
          1029,   102,  2054,  3599,  2003,  1996,  3949,  2005, 10097, 16313,
          2063,  1029,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
def tokenization(example):
    return tokenizer(
        example["question_1"], example["question_2"], return_tensors="pt", padding=True
    )


tokenized_ds = ds.map(tokenization, batched=True)
print(tokenized_ds)

Map: 100%|██████████| 610/610 [00:00<00:00, 5041.63 examples/s]

DatasetDict({
    train: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2438
    })
    test: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 610
    })
})


In [ ]:
tokenized_ds["train"].features

{'dr_id': Value(dtype='int32', id=None),
 'question_1': Value(dtype='string', id=None),
 'question_2': Value(dtype='string', id=None),
 'label': ClassLabel(names=[0, 1], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import wandb

wandb.init(project="medical_bot", name="bert_similarity")

wandb: Currently logged in as: arz-mikhail (arz-mikhail-rtu-mirea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./model",
    # evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    report_to="wandb",  # логирование в W&B
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=lambda p: {
        "accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()
    },
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


ValueError: You have to specify either input_ids or inputs_embeds